# Classifying Objects in Images Using Deep Learning

We used basic neural networks in Chapter 8, Beating CAPTCHAs with Neural Networks.
A recent flood of research in the area has led to a number of significant advances to
that base design. Today, research in neural networks is creating some of the most
advanced and accurate classification algorithms in many areas.
These advances have come on the back of improvements in computational power,
allowing us to train larger and more complex networks. However, the advances are
much more than simply throwing more computational power at the problem.
New algorithms and layer types have drastically improved performance, outside
computational power.

In this chapter, we will look at determining what object is represented in an
image. The pixel values will be used as input, and the neural network will then
automatically find useful combinations of pixels to form higher-level features.
These will then be used for the actual classification. Overall, in this chapter,
we will examine the following:

- Classifying objects in images
- The different types of deep neural networks
- Theano, Lasagne, and Nolearn; libraries to build and train neural networks
- Using a GPU to improve the speed of the algorithms